# Indexing notebook

### First we need to move to the top-level directory.

In [1]:
cd ../..

/Users/leo/SourceTreeGit/FlexNeuART.refact2021


### Lucene indexer options

In [2]:
!scripts/index/create_lucene_index.sh

Using collection root: collections
Specify collection sub-directory, e.g., msmarco_pass (1st arg)
Usage: <collection> [additional options]
Additional options:
-h print help
-exact_match create index for exact match
-index_field indexing field name (default text)
-input_subdir input data sub-directory (default input_data)
-index_subdir index subdirectory (default lucene_index)


By default Lucene using the context of the field `text` to create the full-text index, which is stored in the sub-directory `lucene_index`, but it is possible to create an index for an exact match, use a content of a different field, or store the inde

### Lucene index

In [4]:
!scripts/index/create_lucene_index.sh wikipedia_dpr_nq_sample

Using collection root: collections
Input data directory: collections/wikipedia_dpr_nq_sample/input_data
Index directory:      collections/wikipedia_dpr_nq_sample/lucene_index
Index field name:     text
Exact match param:    
Checking input sub-directory: bitext
Checking input sub-directory: dev
Checking input sub-directory: dev_official
Checking input sub-directory: pass_sample
Found indexable data file: pass_sample/AnswerFields.jsonl.gz
Checking input sub-directory: train_fusion
Found query file: bitext/QuestionFields.jsonl
Found query file: dev/QuestionFields.jsonl
Found query file: dev_official/QuestionFields.jsonl
Found query file: train_fusion/QuestionFields.jsonl
Using the data input file: AnswerFields.jsonl.gz
JAVA_OPTS=-Xms4117329k -Xmx28821303k -server
[main] INFO edu.cmu.lti.oaqa.flexneuart.apps.LuceneIndexer - Creating a new Lucene index, maximum # of docs to process: 2147483647 index field name: text exact match? false
[main] INFO edu.cmu.lti.oaqa.flexneuart.apps.LuceneInde

### Forward indices

A forward index allows one to retrieve an original or parsed field content using a document identifier. The forward index can be stored using two foramts and two storage engines (mapdb and lucene). In most cases, default settings (we use `mapdb` directly) work well. However, by default the index is built in memory. When there is not enough memory, one can build using an option `-fwd_index_type offsetDict`, which stores data records in a separate file and uses `mapdb` to store only offsets & lengths.

There are four types of the field, which include:
1. two parsed textual field formats: 
    - Parsed text without positional information (bag-of-words): **parsedBOW**   
    - Parsed text with positional information: **parsedText**
2. original/unparsed/raw text: **textRaw**
3. binary (can be anything): **binary**

The options are printed by the indexing script:

In [5]:
!scripts/index/create_fwd_index.sh 

Using collection root: collections
collection sub-directory, e.g., msmarco_pass (1st arg)
Usage: <collection> <field definition: examples: text:parsedBOW, text_unlemm:parsedText, text_raw:textRaw, dense_embed:binary> [additional options]
Additional options:
-h print help
-clean remove the previous index
-input_subdir input data sub-directory (default input_data)
-index_subdir index subdirectory (default forward_index)
-fwd_index_type forward index type: dataDict, offsetDict
-fwd_index_store_type a forward backend storage type: lucene, mapdb
-expect_doc_qty expected # of documents in the index


Here we create indices sequentially, but they can also be created **in parallel** (independently for each field):

In [ ]:
!for field_def in dense:binary text:parsedText \
                  text_unlemm:parsedText \
                  title:parsedBOW \
                  text_bert_tok:parsedText \
                  text_raw:textRaw ; do \
 scripts/index/create_fwd_index.sh wikipedia_dpr_nq_sample $field_def ; \
done